This notebook is used for deployment and execution of the S2 cloud free best pixel workflow.

Credentials: create a `.env` file in the root of this repository with the following content:

```
ADES_USER="username"
ADES_TOKEN="token"
```


In [1]:
import json
import os
import time

import dotenv

import pyeodh

dotenv.load_dotenv()

pyeodh.set_log_level(10)

client = pyeodh.Client(
    username=os.getenv("ADES_USER"),
    token=os.getenv("ADES_TOKEN"),
)
ades = client.get_ades()

Run the following cell if deploying the workflow.


In [6]:
cwl_yaml = r"""cwlVersion: v1.0
$graph:
- class: CommandLineTool
  id: 's2_make_stac'
  inputs:
  - id: 'files'
    doc: FILES
    type:
      type: array
      items: File
  - id: 'geometry'
    inputBinding:
      prefix: --geometry
    type:
    - 'null'
    - string
  requirements:
  - class: DockerRequirement
    dockerPull: ghcr.io/eo-datahub/user-workflows/s2_make_stac@sha256:1c5f2166ca78c561caf2db45f7974649d3ad4423f774eda4b5d73d5c96d955ed
  - class: InlineJavascriptRequirement
  doc: "None\n"
  baseCommand:
  - python
  - /app/app.py
  outputs:
  - id: 'stac_catalog'
    outputBinding:
      glob: .
    type: Directory
- class: CommandLineTool
  id: 's2_mosaic'
  inputs:
  - id: 'all_images'
    doc: ALL_IMAGES
    type:
      type: array
      items: File
  - id: 'intersects'
    inputBinding:
      prefix: --intersects
    type:
    - 'null'
    - string
  - id: 'month_json'
    inputBinding:
      prefix: --month-json
    type: File
  outputs:
  - id: 'best_pixel'
    outputBinding:
      glob: '*.tif'
    type: File
  requirements:
  - class: DockerRequirement
    dockerPull: ghcr.io/eo-datahub/user-workflows/s2_mosaic@sha256:d66c855007e6838e493d5f4273ff7867dfe17868f84a66e69728b997e0d3031d
  - class: InlineJavascriptRequirement
  doc: "None\n"
  baseCommand:
  - /usr/local/bin/_entrypoint.sh
  - python
  - /app/app.py
- class: CommandLineTool
  id: 's2_rm_cloud'
  inputs:
  - id: 'item_url'
    inputBinding:
      prefix: --item-url
    type:
    - 'null'
    - string
  outputs:
  - id: 'cloud_masked'
    outputBinding:
      glob: '*.tif'
    type: File
  requirements:
  - class: DockerRequirement
    dockerPull: ghcr.io/eo-datahub/user-workflows/s2_rm_cloud@sha256:c4d602948cd0433142ac26a822f72ab23ad7195ab840d9c2acba673346e6f3d1
  - class: InlineJavascriptRequirement
  doc: "None\n"
  baseCommand:
  - /usr/local/bin/_entrypoint.sh
  - python
  - /app/app.py
- class: CommandLineTool
  id: 's2_search'
  inputs:
  - id: 'catalog'
    inputBinding:
      prefix: --catalog
    type:
    - 'null'
    - string
  - id: 'collection'
    inputBinding:
      prefix: --collection
    type:
    - 'null'
    - string
  - id: 'end_datetime'
    inputBinding:
      prefix: --end-datetime
    type:
    - 'null'
    - string
  - id: 'intersects'
    inputBinding:
      prefix: --intersects
    type:
    - 'null'
    - string
  - id: 'start_datetime'
    inputBinding:
      prefix: --start-datetime
    type:
    - 'null'
    - string
  outputs:
  - id: 'months'
    outputBinding:
      glob: month_*.json
    type:
      items: File
      type: array
  - id: 'urls'
    outputBinding:
      glob: urls.txt
      loadContents: true
      outputEval: $(self[0].contents.split('\n'))
    type:
      items: string
      type: array
  requirements:
  - class: DockerRequirement
    dockerPull: ghcr.io/eo-datahub/user-workflows/s2_search@sha256:1a56d9f39b4281a1662de450cd36414c32bc28ff3b544cbf54a84f92e7c59133
  - class: InlineJavascriptRequirement
  doc: "None\n"
  baseCommand:
  - python
  - /app/app.py
- class: Workflow
  id: 'cloud-free-best-pixel-h'
  inputs:
  - id: 'catalog'
    label: Catalog path
    doc: Full catalog path
    default: supported-datasets/ceda-stac-catalogue
    type: string
  - id: 'collection'
    label: collection id
    doc: collection id
    default: sentinel2_ard
    type: string
  - id: 'intersects'
    label: Intersects
    doc: "a GeoJSON-like json string, which provides a \"type\" member describing
      the type of the geometry and \"coordinates\"  member providing a list of coordinates.
      Will search for images intersecting this geometry.\n"
    default: "{\n  \"type\": \"Polygon\",\n  \"coordinates\": [\n    [\n      [0.08905898091569497,
      52.69722175598818],\n      [0.08905898091569497, 52.15527412683906],\n     \
      \ [0.9565339502005088, 52.15527412683906],\n      [0.9565339502005088, 52.69722175598818],\n\
      \      [0.08905898091569497, 52.69722175598818]\n    ]\n  ]\n}\n"
    type: string
  - id: 'start_datetime'
    label: Start datetime
    doc: Start datetime
    default: '2023-04-01'
    type: string
  - id: 'end_datetime'
    label: End datetime
    doc: End datetime
    default: '2023-06-30'
    type: string
  outputs:
  - id: 'stac_output'
    outputSource:
    - 's2_make_stac/stac_catalog'
    type: Directory
  requirements:
  - class: ResourceRequirement
    coresMin: 3
    ramMin: 15000
  - class: ScatterFeatureRequirement
  label: Cloud free best pixel
  doc: Generate cloud free best pixel mosaic on a per month basis
  steps:
  - id: 's2_search'
    in:
    - id: 'catalog'
      source: 'catalog'
    - id: 'collection'
      source: 'collection'
    - id: 'intersects'
      source: 'intersects'
    - id: 'start_datetime'
      source: 'start_datetime'
    - id: 'end_datetime'
      source: 'end_datetime'
    out:
    - id: 'urls'
    - id: 'months'
    run: '#s2_search'
  - id: 's2_rm_cloud'
    in:
    - id: 'item_url'
      source: 's2_search/urls'
    out:
    - id: 'cloud_masked'
    run: '#s2_rm_cloud'
    scatter:
    - 'item_url'
    scatterMethod: dotproduct
  - id: 's2_mosaic'
    in:
    - id: 'intersects'
      source: 'intersects'
    - id: 'month_json'
      source: 's2_search/months'
    - id: 'all_images'
      source: 's2_rm_cloud/cloud_masked'
    out:
    - id: 'best_pixel'
    run: '#s2_mosaic'
    scatter:
    - 'month_json'
    scatterMethod: dotproduct
  - id: 's2_make_stac'
    in:
    - id: 'geometry'
      source: 'intersects'
    - id: 'files'
      source: 's2_mosaic/best_pixel'
    out:
    - id: 'stac_catalog'
    run: '#s2_make_stac'
"""

Run the following cell if deploying the workflow.


In [ ]:
try:
    ades.get_process("cloud-free-best-pixel-h").delete()
except Exception:
    print("Process not found, no need to undeploy.")

process = ades.deploy_process(cwl_yaml=cwl_yaml)

The following cell is used for execution of the workflow.


In [ ]:
intersects = {
    "type": "Polygon",
    "coordinates": [
        [
            [0.08905898091569497, 52.69722175598818],
            [0.08905898091569497, 52.15527412683906],
            [0.9565339502005088, 52.15527412683906],
            [0.9565339502005088, 52.69722175598818],
            [0.08905898091569497, 52.69722175598818],
        ]
    ],
}

j = ades.get_process("cloud-free-best-pixel-h").execute(
    {
        "catalog": "supported-datasets/ceda-stac-catalogue",
        "collection": "sentinel2_ard",
        "intersects": json.dumps(intersects),
        "start_datetime": "2023-04-01",
        "end_datetime": "2023-06-30",
    }
)
print(j.self_href)